In [1]:
import wandb

import wandb

wandb.login(key='c3dfe55fc29b00c7513f66eaa2f85d87ddde467f')

wandb.init(project="MachineTransalation-en-vi-3")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khaihoang004 (khaihoang004-hanoi-university-of-science-and-technology). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241215_013003-s0l55qc5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dry-vortex-13
wandb: ⭐️ View project at https://wandb.ai/khaihoang004-hanoi-university-of-science-and-technology/MachineTransalation-en-vi-3
wandb: 🚀 View run at https://wa

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
import sentencepiece as spm
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader



In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LEN = 64
MODEL_NAME = "t5-small"
RANDOM_STATE = 42
DATA_PERCENTAGE = 0.35
EN_DATAPATH = r'/kaggle/input/machinetranslation-en-vi/train.en.txt'
VI_DATAPATH = r'/kaggle/input/machinetranslation-en-vi/train.vi.txt'

In [4]:
torch.cuda.empty_cache()

if torch.cuda.is_available():
    print("CUDA is available. Listing all devices:")
    for device_id in range(torch.cuda.device_count()):
        print(f"Device {device_id}: {torch.cuda.get_device_name(device_id)}")
        print(f" - Total Memory: {torch.cuda.get_device_properties(device_id).total_memory / 1e9:.2f} GB")
        print(f" - CUDA Capability: {torch.cuda.get_device_properties(device_id).major}.{torch.cuda.get_device_properties(device_id).minor}")
else:
    print("CUDA is not available on this system.")


CUDA is available. Listing all devices:
Device 0: Tesla T4
 - Total Memory: 15.83 GB
 - CUDA Capability: 7.5
Device 1: Tesla T4
 - Total Memory: 15.83 GB
 - CUDA Capability: 7.5


In [5]:
# Tải hai tokenizer khác nhau
en_tokenizer = T5Tokenizer.from_pretrained("t5-small")
vi_tokenizer = spm.SentencePieceProcessor(model_file = r'/kaggle/input/machinetranslation-en-vi/vi_tokenizer_32128.model')


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
input_text_en = "Hello, how are you?"
input_ids_en = en_tokenizer(input_text_en, return_tensors="pt").input_ids

input_text_vi = "Xin chào, bạn khỏe không?"
input_ids_vi = vi_tokenizer.encode(input_text_vi, out_type=int)

decoded_en = en_tokenizer.decode(input_ids_en[0], skip_special_tokens=True)
decoded_vi = vi_tokenizer.decode(input_ids_vi)

print(f"Input EN: {input_text_en}")
print(f"Input Vi (Tokenized): {input_ids_vi}")
print(f"Decoded Vi: {decoded_vi}")

Input EN: Hello, how are you?
Input Vi (Tokenized): [3264, 1655, 2, 88, 204, 15, 4]
Decoded Vi: Xin chào ⁇  bạn khỏe không?


In [7]:
import random
with open(EN_DATAPATH, 'r', encoding='utf-8') as f_en:
    train_en = f_en.readlines()

with open(VI_DATAPATH, 'r', encoding='utf-8') as f_vi:
    train_vi = f_vi.readlines()

random.seed(RANDOM_STATE)

data_size = len(train_en)
indices = list(range(data_size))
random.shuffle(indices)

train_size = int(DATA_PERCENTAGE * data_size)
val_size = 8000

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]

train_en_subset = [train_en[i] for i in train_indices]
train_vi_subset = [train_vi[i] for i in train_indices]

val_en_subset = [train_en[i] for i in val_indices]
val_vi_subset = [train_vi[i] for i in val_indices]

print(train_size)
print(val_size)

104229
8000


In [8]:
class TranslationDataset(Dataset):
    def __init__(self, source_texts, target_texts, en_tokenizer, vi_tokenizer, max_length=MAX_LEN):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.en_tokenizer = en_tokenizer
        self.vi_tokenizer = vi_tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx]
        target_text = self.target_texts[idx]

        # Input (EN)
        source_encoding = self.en_tokenizer(source_text, truncation=True, padding="max_length", max_length=self.max_length)
        
        # Output (VI)
        target_encoding = self.vi_tokenizer.encode(target_text, out_type=int)
        target_encoding = target_encoding[:self.max_length]
        target_encoding += [self.vi_tokenizer.pad_id()] * (self.max_length - len(target_encoding))  # Pad to have length = max_length
        
        target_attention_mask = [1] * len(target_encoding)                                          # Attention mask for target
        target_attention_mask += [0] * (self.max_length - len(target_attention_mask))               # Pad the attention mask

        return {
            'input_ids': torch.tensor(source_encoding['input_ids']),
            'attention_mask': torch.tensor(source_encoding['attention_mask']),
            'labels': torch.tensor(target_encoding),
            'target_attention_mask': torch.tensor(target_attention_mask),
        }


In [9]:
train_dataset = TranslationDataset(train_en_subset, train_vi_subset, en_tokenizer, vi_tokenizer)
val_dataset = TranslationDataset(val_en_subset, val_vi_subset, en_tokenizer, vi_tokenizer)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
model.to(device)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [11]:
# Training arguments for resuming training
training_args = TrainingArguments(
    output_dir='./results_v1',          # output directory
    evaluation_strategy="epoch",        # evaluate at the end of each epoch
    save_total_limit=1,
    logging_dir='./logs',               # directory for storing logs
    logging_steps=100,
    fp16=True,
    push_to_hub=False,                  # Disable Hugging Face Hub interaction
    report_to="wandb",
    save_strategy="epoch",              # save at the end of each epoch
    save_steps=500,
    load_best_model_at_end=True,
    dataloader_num_workers=4,

    learning_rate=5e-4,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    num_train_epochs=25,                 # số epoch mới để train thêm
    weight_decay=0.1,                   
    metric_for_best_model="eval_loss",
    lr_scheduler_type="linear",
    resume_from_checkpoint=True
)

# Trainer setup
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Resume training
trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Passing a tuple of

Epoch,Training Loss,Validation Loss
1,1.833300,1.695019
2,1.528300,1.389206
3,1.373300,1.241083
4,1.262200,1.152041
5,1.187800,1.094445
6,1.128600,1.051417
7,1.095100,1.021713
8,1.055100,0.990982
9,1.025900,0.970822
10,1.000100,0.956018


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(

TrainOutput(global_step=10200, training_loss=1.0475931541592467, metrics={'train_runtime': 12913.6973, 'train_samples_per_second': 201.78, 'train_steps_per_second': 0.79, 'total_flos': 4.40829394550784e+16, 'train_loss': 1.0475931541592467, 'epoch': 25.0})

In [12]:
def translate_text(input_text, model, en_tokenizer, vi_tokenizer, device):
    inputs = en_tokenizer(input_text, return_tensors="pt").to(device)
    print(inputs)
    outputs = model.generate(inputs['input_ids'], max_length=MAX_LEN)
    print(f"Generated output shape: {outputs[0]}")
    
    translated_text = vi_tokenizer.decode(outputs[0].tolist())
    return translated_text

# Test
test_text_1 = "What is your name?"
translation_1 = translate_text(test_text_1, model, en_tokenizer, vi_tokenizer, DEVICE)

print(f"English: {test_text_1}")
print(f"Vietnamese Translation: {translation_1}")

# Test
test_text_2 = "How are you?"
translation_2 = translate_text(test_text_2, model, en_tokenizer, vi_tokenizer, DEVICE)

print(f"English: {test_text_2}")
print(f"Vietnamese Translation: {translation_2}")

test_text_3 = "I go to school."
translation_3 = translate_text(test_text_3, model, en_tokenizer, vi_tokenizer, DEVICE)

print(f"English: {test_text_3}")
print(f"Vietnamese Translation: {translation_3}")

test_text_4 = "Go home now!"
translation_4 = translate_text(test_text_4, model, en_tokenizer, vi_tokenizer, DEVICE)

print(f"English: {test_text_4}")
print(f"Vietnamese Translation: {translation_4}")

{'input_ids': tensor([[363,  19,  39, 564,  58,   1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}
Generated output shape: tensor([   0, 5007,  112,   12,  365,    6,    4,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0], device='cuda:0')
English: What is your name?
Vietnamese Translation: Tên anh là gì ?
{'input_ids': tensor([[571,  33,  25,  58,   1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}
Generated output shape: tensor([  0, 449, 160, 241,   6,   4,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          

In [13]:
# model.save_pretrained('./fine_tuned_t5_translation')
# en_tokenizer.save_pretrained('./fine_tuned_t5_translation')
